In [ ]:
# default_exp core.__init__

# Core components

## Imports

Python modules

In [ ]:
# export
import json
import os
import pathlib
import re
from dataclasses import dataclass
from datetime import datetime, timezone
from enum import Enum
from textwrap import dedent
from typing import Pattern, List

Third-party modules

In [ ]:
# export
import dotenv
from requests import Session

Development modules

In [ ]:
# hide
from nbdev.showdoc import show_doc

## Core functions

In [ ]:
# export
NULL_CURSOR = json.dumps(None)

In [ ]:
# export
def build_query(query_string: str, **kwargs) -> str:
    return json.dumps({"query": dedent(query_string.strip()) % kwargs})

In [ ]:
# export
def print_response(response, compact: bool = False):
    print(
        json.dumps(
            response.json(),
            indent=None if compact else 4,
        )
    )

In [ ]:
# export
def get_response_data(
    response, key_path: str | None = None, action: str = "actor"
) -> dict | None:
    data = response.json().get("data").get(action)

    if key_path is not None:
        for key in key_path.split("."):
            if key.isdecimal() and isinstance(data, list):
                data = data[int(key)]
            else:
                data = data.get(key)

    return data

In [ ]:
# export
def snake2camel(snake_string: str) -> str:
    head, *tail = snake_string.split("_")
    camel_string = "".join([head.lower(), *[word.capitalize() for word in tail]])
    return camel_string

In [ ]:
# export
def camel2snake(camel_string: str) -> str:
    return re.sub(r"([A-Z][a-z]+)", r"_\1", camel_string).lower()

In [ ]:
# export
def camelize_keys(obj: dict) -> dict:
    camelized_obj = {}

    for key, value in dict(obj).items():
        if isinstance(value, dict):
            camelized_obj.update(
                {
                    snake2camel(key): camelize_keys(value),
                },
            )
        elif isinstance(value, list) and len(value) > 0:
            camelized_obj.update(
                {
                    snake2camel(key): [
                        camelize_keys(item) if isinstance(item, dict) else item
                        for item in value
                    ],
                },
            )
        else:
            camelized_obj.update(
                {
                    snake2camel(key): value,
                },
            )

    return camelized_obj

In [ ]:
# export
def snakeize_keys(obj: dict) -> dict:
    snakeized_obj = {}

    for key, value in dict(obj).items():
        if isinstance(value, dict):
            snakeized_obj.update(
                {
                    camel2snake(key): snakeize_keys(value),
                },
            )
        elif isinstance(value, list) and len(value) > 0:
            snakeized_obj.update(
                {camel2snake(key): [snakeize_keys(item) for item in value]},
            )
        else:
            snakeized_obj.update(
                {
                    camel2snake(key): value,
                }
            )

    return snakeized_obj

## Core Classes

### Utilitary classes 

In [ ]:
# export
class EntityEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Enum):
            return o.value

        if isinstance(o, datetime):
            # Accoirding to newrelic dates are handled in
            # ISO8601 whithout microseconds and with Z as
            # time zone but python datetime.isoformat
            # doesn't workt in such way.
            # return obj.isoformat()
            # The user must be responsible for provide
            # obj in UTC timezone
            return o.strftime("%Y-%m-%dT%H:%M:%SZ")

        if hasattr(o, "__dict__"):
            return o.__dict__

        return super().default(o)

In [ ]:
# hide
show_doc(EntityEncoder)

### Core Mixins

In [ ]:
# export
class JSONMixin:
    property_processors: dict | None = None

    @classmethod
    def _process_property(cls, property_name: str, json_str: str):
        processor = cls.property_processors.get(property_name)
        return processor(json_str)

    @classmethod
    def _process_properties(cls, json_obj: dict):
        for property_name, processor in (cls.property_processors or {}).items():
            if property_name in json_obj:
                property_obj = json_obj[property_name]

                if isinstance(property_obj, list):
                    json_obj[property_name] = [
                        processor(json.dumps(item)) for item in property_obj
                    ]
                elif property_obj is not None:
                    json_obj[property_name] = processor(json.dumps(property_obj))

        return json_obj

    @classmethod
    def _load_json(cls, json_str: str) -> dict:
        json_obj = json.loads(json_str)

        if isinstance(json_obj, dict):
            json_obj = snakeize_keys(json_obj)
            json_obj = cls._process_properties(json_obj)

        return json_obj

    @staticmethod
    def _dumps_json(obj: dict, **kwargs) -> str:
        json_str = json.dumps(obj, cls=EntityEncoder)

        json_obj = json.loads(json_str)
        json_obj = camelize_keys(json_obj)

        json_str = json.dumps(json_obj, **kwargs)

        return json_str

    @classmethod
    def from_json(cls, json_str: str):
        json_obj = cls._load_json(json_str)

        if isinstance(json_obj, dict):
            return cls(**json_obj)

        return cls(json_obj)

    def to_json(self, **kwargs) -> str:
        return self._dumps_json(self.__dict__, **kwargs)

In [ ]:
# hide
show_doc(JSONMixin)

In [ ]:
# export
class GQLMixin:
    JSON_KEY_REGEX: Pattern[str] = re.compile(r'"([^ "]+)":')
    INPUT_PROPERTIES: List[str] | None = None

    def get_gql_input(self):
        pass

    @property
    def gql(self) -> str:
        gql_str = self.to_json()
        gql_str = self.JSON_KEY_REGEX.sub(r"\1:", gql_str)
        # gql_str = gql_str.replace(
        #     f'"{DashboardPermission.PRIVATE.value}"', DashboardPermission.PRIVATE.value
        # )
        # gql_str = gql_str.replace(
        #     f'"{DashboardPermission.PUBLIC_READ_ONLY.value}"',
        #     DashboardPermission.PUBLIC_READ_ONLY.value,
        # )
        # gql_str = gql_str.replace(
        #     f'"{DashboardPermission.PUBLIC_READ_WRITE.value}"',
        #     DashboardPermission.PUBLIC_READ_WRITE.value,
        # )

        return gql_str

In [ ]:
# export
class SerializableMixin(GQLMixin, JSONMixin):
    pass

### Base Classes

In [ ]:
# export
class SerializableEnum(SerializableMixin, Enum):
    def to_json(self, **kwargs) -> str:
        return json.dumps(self.value, **kwargs)

    def __repr__(self):
        return f"{self.__class__.__name__}.{self.name}"

In [ ]:
# export
class BaseEntity(SerializableMixin):
    pass